In [4]:
import re
test1 = "N4d2N0a2m2s1MUL"
test2 = "N3m3m5N2m5ADDN1a4m2s5DIV"
match = lambda x: re.findall(r'N[a-z0-9]+|V[a-z0-9]+|[A-Z]{3}', x)
terms = match(test2)
print(terms)


['N3m3m5', 'N2m5', 'ADD', 'N1a4m2s5', 'DIV']


In [384]:
def extract_variable_name(code):
  return code[::2]

def process_number_operation(expr):
    value = 0
    current_operation = 'ADD'

    # Patrón para identificar números y operaciones
    tokens = re.findall(r'[a-zA-Z]+|\d+', expr)
    
    for token in tokens:
        if token.isdigit():
            num = int(token)
            if current_operation == 'ADD':
                value += num
            elif current_operation == 'SUB':
                value -= num
            elif current_operation == 'MUL':
                value *= num
            elif current_operation == 'DIV' and num != 0:
                value //= num
        else:
            if token == 'a':
                current_operation = 'ADD'
            elif token == 's':
                current_operation = 'SUB'
            elif token == 'm':
                current_operation = 'MUL'
            elif token == 'd':
                current_operation = 'DIV'
    
    return value

def process_string(string_code):
    return ''.join(reversed(string_code[::2]))

def process_value(element):
    if element.startswith('B'):
        string_code = element[1:-1]
        return process_string(string_code)
    elif element.startswith('N'):  
        return process_number_operation(element[:])
    else: 
        variable_name = extract_variable_name(element[1:-1])
        return f'var[{variable_name}]'

In [385]:
test_st = "Vvrarrr1rN0a2m2s1MUL"
test_st.rsplit("V",1)
terms = match(test_st)
terms

['Vvrarrr1r', 'N0a2m2s1', 'MUL']

In [386]:
terms[0][1:-1:2]

'var1'

In [10]:
cond1 = "HOB HOB HO BebnbobtbobnbP  | N1s0N1s0QE OH B1bP  | VfrlrargrN1s0QE BOH  BsbibgbablbfbP  | VfrlrargrN1s0EG BOH"
cond2 = "HOB HO HOB HO VnrurmrbrP  | N1s1N1s1QE OH  VgrrrerartrerrrP  | VgrrrerartrerrrVnrurmrbrTG BOH  | N1s1N1s1QE OH HOB HO VnrurmrbrP  | N1s1N1s1QE OH Vvrarrr1rP  | Vvrarrr1rVnrurmrbrTG BOH  | Vvrarrr1rVgrrrerartrerrrTG  BOH"
cond3 = "HOB HOB BdbebibfbibdbobmbBebsbabrbhbpbADDVprhrrrarsrer=  | VmrordrirfryrN1s0EG BOH  | VprhrrrarsrerBlblbubnbEN AND VmrordrirfryrN1s1EG BOH"
loop1 = "POOL Vvrarrr1rN1s0ADDVvrarrr1r=  Vvrarrr2rP  Vvrarrr1rN0a1m2a1ADDVvrarrr2r=  | Vvrarrr1rN1a0m2a3TL LOOP"
loop2 = "POOL HOB VkrzreryrVwrxrzr6rcrzr=  | Vsrur7r2rdrzr9rzrcrVfr7rwrurEN BOH  Vwrwr1rordrkrN1s0ADDVwrwr1rordrkr=  Vqrer2rqrVvr9rkrurlr5r6ryrgr=  POOL Vyrzrkr4rN1s0ADDVyrzrkr4r=  N2Vwrxrzr6rcrzr=  VkrzreryrP  | Vyrzrkr4rN2m2m2m2m2d1EL LOOP  N2m2m2m2m2d1Vyrzrkr4r=  N2m2m2P  | Vwrwr1rordrkrN8a24m2s27TL LOOP"

match = lambda x, y: re.findall(r'[A-Za-z0-9_=.]+(?: OR [A-Za-z0-9_]+| AND [A-Za-z0-9_]+)?|BOH|HOB|OH|HO|[A-Za-z0-9=_]+|\|', x[y:-y])

#tokens_test = re.findall(r'[A-Za-z0-9= ]+  \| [A-Za-z0-9]+(?: OR [A-Za-z0-9]+| AND [A-Za-z0-9]+)?', test)
tokens = match(loop2, 4)
tokens

['HOB',
 'VkrzreryrVwrxrzr6rcrzr=',
 '|',
 'Vsrur7r2rdrzr9rzrcrVfr7rwrurEN',
 'BOH',
 'Vwrwr1rordrkrN1s0ADDVwrwr1rordrkr=',
 'Vqrer2rqrVvr9rkrurlr5r6ryrgr=',
 'POOL',
 'Vyrzrkr4rN1s0ADDVyrzrkr4r=',
 'N2Vwrxrzr6rcrzr=',
 'VkrzreryrP',
 '|',
 'Vyrzrkr4rN2m2m2m2m2d1EL',
 'LOOP',
 'N2m2m2m2m2d1Vyrzrkr4r=',
 'N2m2m2P',
 '|',
 'Vwrwr1rordrkrN8a24m2s27TL']

In [11]:
def get_condition_structure(tokens):
  g_stack = []
  stack = []
  counter = 0

  is_condition = False
  is_loop = False

  for element in tokens:
    if element == 'BOH' and not is_loop:
      counter += 1
      stack.append(element)
      stack.append(" ")
    elif element == 'HOB'and not is_loop:
      counter -= 1
      stack.append(element)
      stack.append(" ")
    elif element == 'LOOP' and not is_condition:
      counter += 1
      stack.append(element)
      stack.append(" ")
    elif element == 'POOL'and not is_condition:
      counter -= 1
      stack.append(element)
      stack.append(" ")
    elif counter != 0:
      stack.append(element)
      stack.append(" ")
    if counter == 0:
      is_condition = False
      is_loop = False
      if len(stack) > 0:
        g_stack.append(''.join(stack).strip())
        stack = []
      else:
        g_stack.append(element)
  return g_stack

get_condition_structure(tokens)

['HOB VkrzreryrVwrxrzr6rcrzr= | Vsrur7r2rdrzr9rzrcrVfr7rwrurEN BOH',
 'Vwrwr1rordrkrN1s0ADDVwrwr1rordrkr=',
 'Vqrer2rqrVvr9rkrurlr5r6ryrgr=',
 'POOL Vyrzrkr4rN1s0ADDVyrzrkr4r= N2Vwrxrzr6rcrzr= VkrzreryrP | Vyrzrkr4rN2m2m2m2m2d1EL LOOP',
 'N2m2m2m2m2d1Vyrzrkr4r=',
 'N2m2m2P',
 '|',
 'Vwrwr1rordrkrN8a24m2s27TL']

In [389]:
def evaluate_condition(cond):
  option1, option2, operation = re.findall(r'N[a-z0-9]+|V[a-z0-9]+|B[a-z0-9]+|[A-Z]{2}', cond)
  print(option1, option2, operation)
# EN >
# EG >=
# EL <=
# QE ==
# TG !=
# TL <
  print(process_value(option1), end="")
  if operation == 'EN':
    print("!=", end="")
  elif operation == 'EG':
    print(">=", end="")
  elif operation == 'EL':
    print("<=", end="")
  elif operation == 'QE':
    print("==", end="")
  elif operation == 'TG':
    print(">", end="")
  else:
    print("<", end="")
  print(process_value(option2))

  return True

In [390]:
def execute_condition(condition):
  if "OR" in condition:
    cond1 , cond2 = condition.split(" OR ")
    return evaluate_condition(cond2) or evaluate_condition(cond1)
  elif "AND" in condition:
    cond1 , cond2 = condition.split(" AND ")
    return evaluate_condition(cond2) and evaluate_condition(cond1)
  else:
    return evaluate_condition(condition)

def transform_to_action(line):
  print(f'Line {line} executed')

In [398]:
def check_condition(cond):
  condition = match(cond)
  conditions = get_condition_structure(condition)
  print(conditions)

  actions=[]
  is_condition = True
  do_actions = False
  look_next_condition = False

  for element in reversed(conditions):
    if is_condition and not look_next_condition:
      do_actions = execute_condition(element)
      if do_actions : 
        is_condition = False
        actions = []
      else:
        look_next_condition = True
    else:
      if element == "OH":
        if do_actions : break
        is_condition = True
        look_next_condition = False
      elif do_actions and element != "|" and element != "HO":
        actions.append(element)

  print(f'Executing {actions}')

  for action in actions:
    if "BOH" in action: 
      # print(action)
      check_condition(action)
    else:
      transform_to_action(action)
    


In [20]:
def check_loop(loop):
  print(loop)
  sintaxis = get_condition_structure(match(loop,4))
  
  condition = ""
  actions = []
  is_condition = True

  for element in reversed(sintaxis):
    if element == "|":
      is_condition = False
    elif is_condition:
      condition = element
    else:
      actions.append(element)

  print(f'Condition {condition}')
  for action in actions:
    print(action)

check_loop(loop2)

POOL HOB VkrzreryrVwrxrzr6rcrzr=  | Vsrur7r2rdrzr9rzrcrVfr7rwrurEN BOH  Vwrwr1rordrkrN1s0ADDVwrwr1rordrkr=  Vqrer2rqrVvr9rkrurlr5r6ryrgr=  POOL Vyrzrkr4rN1s0ADDVyrzrkr4r=  N2Vwrxrzr6rcrzr=  VkrzreryrP  | Vyrzrkr4rN2m2m2m2m2d1EL LOOP  N2m2m2m2m2d1Vyrzrkr4r=  N2m2m2P  | Vwrwr1rordrkrN8a24m2s27TL LOOP
Condition Vwrwr1rordrkrN8a24m2s27TL
N2m2m2P
N2m2m2m2m2d1Vyrzrkr4r=
POOL Vyrzrkr4rN1s0ADDVyrzrkr4r= N2Vwrxrzr6rcrzr= VkrzreryrP | Vyrzrkr4rN2m2m2m2m2d1EL LOOP
Vqrer2rqrVvr9rkrurlr5r6ryrgr=
Vwrwr1rordrkrN1s0ADDVwrwr1rordrkr=
HOB VkrzreryrVwrxrzr6rcrzr= | Vsrur7r2rdrzr9rzrcrVfr7rwrurEN BOH
